In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import time



In [53]:
url = "https://twitter.com/i/flow/login"

In [38]:
options = Options()
options.add_argument('--headless')
options.add_argument('--headless')
options.add_argument('--headless')
options.add_argument('--headless')
options.add_argument('--headless')
options.add_argument('--headless')


In [47]:
# login , password 
def login_twitter(driver, username, password):
    #open the web page in the brower:
    driver.get("https://twitter.com/i/flow/login")
    
    #find the boxes for username and password
    username_field = driver.find_by_class_name("r-30o5oe r-1dz5y72 r-13qz1uu r-1niwhzg r-17gur6a r-1yadl64 r-deolkf r-homxoj r-poiln3 r-7cikom r-1ny4l3l r-t60dpp r-fdjqy7").click()
    password_field = driver.find_by_class_name("r-30o5oe r-1dz5y72 r-13qz1uu r-1niwhzg r-17gur6a r-1yadl64 r-deolkf r-homxoj r-poiln3 r-7cikom r-1ny4l3l r-t60dpp r-fdjqy7").click()


    #enter your username :
    username_field.send_keys(username)
    
    driver.implicitly_wait(1)
    driver.find_element_by_class_name("css-1rynq56 r-bcqeeo r-qvutc0 r-37j5jr r-q4m81j r-a023e6 r-rjixqe r-b88u0q r-1awozwy r-6koalj r-18u37iz r-16y2uox r-1777fci").click()
    
   

    #enter your password :
    password_field.send_keys(password)
    driver.find_elemnet_by_class_name("css-1qaijid r-bcqeeo r-qvutc0 r-poiln3").click()
    
    
    
    return

In [40]:
# twitter search definition 
class wait_for_more_than_n_elements_to_be_present(object):
    def __init__(self, locator, count):
        self.locator = locator
        self.count = count

    def __call__(self,driver):
        try : 
            elements = EC.find_elements(driver, self.locator)
            return len(elements) > self.count
        except StaleElementReferenceException:
            return False

def search_twitter(driver, query):
    # wait until the search box has loaded :
    box = driver.wait.until(EC.presence_of_all_elements_located((By.NAME , "q")))

    #find the search box in the html : 
    driver.find_element_by_name("q").clear()

    #enter your search string in the search box :
    box.send_keys(query)
    
    # submit the query (like hitting return):
    box.submit()

    # initial wait for the search results to load

    wait = WebDriverWait(driver, 10)

    try:

        # wait until the first search result is found. Search results will be tweets, which are html list items and have the class='data-item-id':

        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li[data-item-id]")))



        # scroll down to the last tweet until there are no more tweets:

        while True:



            # extract all the tweets:

            tweets = driver.find_elements_by_css_selector("li[data-item-id]")



            # find number of visible tweets:

            number_of_tweets = len(tweets)



            # keep scrolling:

            driver.execute_script("arguments[0].scrollIntoView();", tweets[-1])



            try:

                # wait for more tweets to be visible:

                wait.until(wait_for_more_than_n_elements_to_be_present(

                    (By.CSS_SELECTOR, "li[data-item-id]"), number_of_tweets))



            except TimeoutException:

                # if no more are visible the "wait.until" call will timeout. Catch the exception and exit the while loop:

                break



        # extract the html for the whole lot:

        page_source = driver.page_source



    except TimeoutException:



        # if there are no search results then the "wait.until" call in the first "try" statement will never happen and it will time out. So we catch that exception and return no html.

        page_source=None



    return page_source
 

            

In [41]:
# 
def extract_tweets(page_source):
    soup = bs(page_source, 'lxml')
    tweets = []
    for li in soup.find_all("li", class_='css-1qaijid r-bcqeeo r-qvutc0 r-poiln3'):
        # If out li doesn't have a tweet-id , we skip it as it's not going to be a tweet .
        if 'data-item-id' not in li.attrs:
            continue
        else:
            tweet = {'text' : None}
            # tweet text 
            text_p = li.find("p" , class_= "tweet_text")
            if text_p is not None:
                tweet['text'] = text_p.get_text()

            tweets.append(tweet) 
    return tweets


In [48]:
if __name__ == "__main__":
    #start a driver for web brower:
    driver = init_driver()
    #login in to twitter (replace username / password with your own):
    username = "---"
    password = "---"
    login_twitter(driver, username, password)

    #search twitter:
    query = "성수#승연아_우즈야_여기야 OR #신동_넌이미먹었겠지 OR #박지성_단1g도안줌 OR #제노의_맛그당어 OR #런쥔이_밥무거쒀 OR #확신해찬맛집 OR #몬베베가_몬베베에게_추천하는_맛집 OR #백현이를_위한_맛집투어 OR #황제님을위한메뉴판 OR #강다니엘_염염긋 OR #완전_장원영_스타일_아니냐구 OR #ㄷㅂㅇㅈ OR #런쥔이의_맛집내비게이션 OR #승철이_이거먹고_볼빵빵했꾸마 OR #정한아_마니머거여ㅎㅎ OR #조슈아_먹어보슈아 OR #순영아_이거_맛있어 OR #원우야_여기_테이스티 OR #민규라고치고_디너쇼해도돼 OR #도겸이도_도아할_맛집 OR #믿고먹어boo세요"
    page_source = search_twitter(driver, query)

    #extract info from the search results:
    tweets = extract_tweets(page_source)

    close_driver(driver) 

AttributeError: 'WebDriver' object has no attribute 'find_by_class_name'